## Similar Questions search


The idea is that similar questions will have similar answers. So once we can find similar questions then we can just rank there answers ?

Strategy :

1. Search "topics" in question index.

2. get q_n (some number which can yeild 1000 answers) similar questions

3. Now rank these 1000 using RoBERTa-base


In [1]:
import glob
from tqdm.notebook import tqdm
import os
import jsonlines
import re
import xml.etree.ElementTree as ET
from elasticsearch import Elasticsearch
import re
from collections import defaultdict
import numpy as np
from scipy import spatial
import numpy as np

In [2]:
class Topic:
    """
    This class shows a topic for task 1. Each topic has an topic_id which is str, a title and question which
    is the question body and a list of tags.
    """

    def __init__(self, topic_id, title, question, tags):
        self.topic_id = topic_id
        self.title = title
        self.question = question
        self.lst_tags = tags


class TopicReader:
    """
    This class takes in the topic file path and read all the topics into a map. The key in this map is the topic id
    and the values are Topic which has 4 attributes: id, title, question and list of tags for each topic.

    To see each topic, use the get_topic method, which takes the topic id and return the topic in Topic object and
    you have access to the 4 attributes mentioned above.
    """

    def __init__(self, topic_file_path):
        self.__map_topics = self.__read_topics(topic_file_path)

    def __read_topics(self, topic_file_path):
        map_topics = {}
        tree = ET.parse(topic_file_path)
        root = tree.getroot()
        for child in root:
            topic_id = child.attrib['number']
            title = child[0].text
            question = child[1].text
            lst_tag = child[2].text.split(",")
            map_topics[topic_id] = Topic(topic_id, title, question, lst_tag)
        return map_topics

    def get_topic(self, topic_id):
        if topic_id in self.__map_topics:
            return self.__map_topics[topic_id]
        return None

def remove_stop(query):
    with open('englishST.txt') as f:
        all_stopwords = f.readlines()
    # you may also want to remove whitespace characters like `\n` at the end of each line
    all_stopwords = [x.strip() for x in all_stopwords] 
    text_tokens = query.split(' ')
    query = [word for word in text_tokens if not word in all_stopwords]
    query = ' '.join(query)
    return query

def remove_punct(my_str):
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    # To take input from the user
    # my_str = input("Enter a string: ")

    # remove punctuation from the string
    no_punct = ""
    for char in my_str:
        if char not in punctuations:
            no_punct = no_punct + char

    # display the unpunctuated string
    return no_punct

es = Elasticsearch(['http://csxindex05:9200/'], verify_certs=True)
queries = []
#"In this example, the title and the question body of topic with id A.1 is printed."
topic_file_path = "/data/szr207/dataset/ArqMath/Task1/Sample Topics/Task1_Samples_V2.0.xml"
topic_reader = TopicReader(topic_file_path)
dict_q_a = defaultdict(list)
for topic_id in ['A.31','A.78', 'A.101']:
#     topic_id = "A.31"
    title = re.sub('<[^<]+?>', '', topic_reader.get_topic(topic_id).title)
    body = topic_reader.get_topic(topic_id).question
    body_pro = re.sub('<[^<]+?>', '', body)
    query = title + '. ' + body_pro
    queries.append(query)
    query = query.lower()
    query = remove_stop(query)
    
#     print(topic_reader.get_topic(topic_id).lst_tags)
    print(topic_id, topic_reader.get_topic(topic_id).title, topic_reader.get_topic(topic_id).question)
#     print(body_pro)
    print("=================================")
    body = {
        "size": 500,
        "query": {
            "match": {
                "body": query
            }

        }
    }

    res = es.search(index="question_bulk_index", body=body)
    for result in res['hits']['hits']:
        dict_q_a[topic_id].append(result['_source']['post_id'])

A.31 Doubt on Implication of Logical reasoning <p>Its evident that in the truth table of <span class="math-container" id="q_253">$p \to q$</span> </p>  <p>When <span class="math-container" id="q_254">$p$</span> is False and <span class="math-container" id="q_255">$q$</span> is True, Then <span class="math-container" id="q_256">$p \to q$</span> is True.</p>  <p>But in some instances i could not convince myself about this truth value.</p>  <p>For example:</p>  <p><span class="math-container" id="q_257">$p$</span>: Quadrilateral is Cyclic</p>  <p><span class="math-container" id="q_258">$q$</span>: Opposite angles are supplementary</p>  <p>Now is <span class="math-container" id="q_259">$p$</span> is False and <span class="math-container" id="q_260">$q$</span> is True, how can <span class="math-container" id="q_261">$p \to q$</span> can be True?</p> 
A.78 if <span class="math-container" id="q_730">$\sum_{n=1}^{\infty} a_n$</span> converges absolutely then prove that <span class="math-contai

In [3]:
ques_path = '/data/szr207/dataset/ArqMath/jsons/questions/all.ques.jsonl'

dict_ques = {}

with jsonlines.open(os.path.join(ques_path)) as reader:
        for obj in tqdm(reader):
            dict_ques[obj['post_id']] = obj

In [4]:
ans_path = '/data/szr207/dataset/ArqMath/jsons/answers/all.ans.jsonl'
      
dict_ans = {}
    
with jsonlines.open(os.path.join(ans_path)) as reader:
        for obj in tqdm(reader):
            dict_ans[obj['post_id']] = obj

In [5]:
dict_aid_post = {}
with jsonlines.open(os.path.join(ans_path)) as reader:
    for obj in tqdm(reader):
        if obj['body']:
            obj['body'] = re.sub('<[^<]+?>', '',  obj['body'])
            dict_aid_post[obj['post_id']] = obj['body']

In [6]:
count = 0

dict_topic_ans = {}

# accepted answers can have a separate list and get higher votes

for topic_id in dict_q_a:
    list_ans_idx = []
    list_other_ans = []
    for qid in tqdm(dict_q_a[topic_id]):
        if dict_ques[qid]['accepted_answer_id'] in dict_ans:
            list_ans_idx.append(dict_ques[qid]['accepted_answer_id'])
        if dict_ques[qid]['answers']:
            for ans in dict_ques[qid]['answers']:
                if ans in dict_ans:
                    list_other_ans.append(ans)
    list_ans_idx.extend(list_other_ans)
    dict_topic_ans[topic_id] = list(set(list_ans_idx))
    print(len(list_ans_idx))


1330



924



1392


In [7]:
from transformers import pipeline
from transformers import *

feat_ext = pipeline("feature-extraction", model="shauryr/arqmath-roberta-base-1.5M", tokenizer='roberta-base', device=0)
# feat_ext = pipeline("feature-extraction", model="shauryr/arqmath-roberta-base", tokenizer='roberta-base', device=0)
# tokenizer =  BertTokenizer.from_pretrained('bert-base-cased')
# feat_ext = pipeline("feature-extraction", model="roberta-base", tokenizer='roberta-base', device=0)

In [42]:
import gc
del feat_ext
gc.collect()

1151

In [8]:
ques_emb = []
for query in queries:
    ques_emb.append(np.mean(feat_ext(query)[0], axis=0))

In [16]:
def reduce_str(body, N):
    if len(body.split()) > N:
        return ' '.join(body.split()[:N])
    return body

In [9]:
def Sort_Tuple(tup):  
    # reverse = None (Sorts in Ascending order)  
    # key is set to sort using second element of  
    # sublist lambda has been used  
    tup.sort(key = lambda x: x[2])
    return tup

In [11]:
result_list

[[('A.31', 2212466, 0.4776468531746543),
  ('A.31', 2408551, 0.474585469953263),
  ('A.31', 946209, 0.46802374333004537),
  ('A.31', 74459, 0.46327297398584877),
  ('A.31', 2345997, 0.461624814773834),
  ('A.31', 2006909, 0.46129168843964263),
  ('A.31', 2096584, 0.4581155405912364),
  ('A.31', 1586386, 0.4549135116220008),
  ('A.31', 1878292, 0.4530849107557975),
  ('A.31', 2375505, 0.45265103625358694),
  ('A.31', 2118243, 0.45154076320619707),
  ('A.31', 557114, 0.44807752894296204),
  ('A.31', 443860, 0.44711014878371813),
  ('A.31', 443885, 0.4462368198837663),
  ('A.31', 960140, 0.44144386306233896),
  ('A.31', 696027, 0.4404310500881681),
  ('A.31', 650293, 0.4384860334709949),
  ('A.31', 2253476, 0.4365837659052674),
  ('A.31', 1179430, 0.4348915538227309),
  ('A.31', 808072, 0.4342626419744098),
  ('A.31', 915761, 0.43371642031010005),
  ('A.31', 2386894, 0.43246011201875123),
  ('A.31', 2946119, 0.4316873618349625),
  ('A.31', 440062, 0.4308452161469958),
  ('A.31', 1680459, 

In [10]:
dict_q_idx = {'A.31':0,'A.78':1, 'A.101':2}
result_list = []
count = 0
import math
for qid in ['A.31','A.78', 'A.101']:
    tup_postid_sim = []
    for post_id in tqdm(dict_topic_ans[qid]):
#         print(qid,post_id)
#         ans_emb = np.mean(feat_ext(reduce_str(dict_aid_post[post_id],300))[0], axis=0)
#         ans_emb = np.mean(feat_ext(dict_aid_post[post_id])[0], axis=0)
        ans_emb = feat_ext(dict_aid_post[post_id])[0][0]
#         print(ans_emb)
        result = 1 - spatial.distance.cosine(ques_emb[dict_q_idx[qid]], ans_emb)
        if math.isnan(result):
            count+=1
            print(qid, post_id, count)
#             tup_postid_sim.append((qid,post_id,str(1.0)))
            continue
        tup_postid_sim.append((qid,post_id,result))
    
    print(len(tup_postid_sim))
    print(qid)
    result_list.append(Sort_Tuple(tup_postid_sim)[::-1][:1000])
#         result = 1 - spatial.distance.cosine(ques_emb[idx], ans_emb[dict_aid_idx[int(post_id)]])
#         tup_postid_sim.append((q_id,post_id,result))
#     print(len(tup_postid_sim))
#     print(q_id)
# #     print(Sort_Tuple(tup_postid_sim)[::-1][:1000][:4])
#     result_list.append(Sort_Tuple(tup_postid_sim)[::-1][:100])


1033
A.31



640
A.78



1072
A.101


In [15]:
with open('es.500.sim.ques.fintune.1.5M.dat', 'w') as eval_file:
    for res in result_list:
        count = 1
        for tuples in res:
#             print(tuples)
            eval_file.write(tuples[0]+'\t'+ '1\t' + str(tuples[1]) +'\t'+str(count)+'\t'+ str(tuples[2])+'\t'+ 'rr'+'\n')
#             print(tuples[0]+'\t'+ '1\t' + tuples[1] +'\t'+str(count)+'\t'+ str(tuples[2])+'\t'+ 'rr')
            count+=1

In [36]:
# dict_topic_ans
with open('es.500.sim.baseline.dat', 'w') as eval_file:
    for res in dict_topic_ans:
        count = 1
        score = 2000
        for idx in dict_topic_ans[res]:
#             print(tuples)
            eval_file.write(res+'\t'+ '1\t' + str(idx) +'\t'+str(count)+'\t'+ str(score)+'\t'+ 'sim_base'+'\n')
#             print(tuples[0]+'\t'+ '1\t' + tuples[1] +'\t'+str(count)+'\t'+ str(tuples[2])+'\t'+ 'rr')
            count+=1
            score-=1

In [33]:
dict_topic_ans

{'A.31': [1587200,
  292868,
  692232,
  2883596,
  1298448,
  2408464,
  2117649,
  1290258,
  1587220,
  440341,
  1298454,
  98328,
  931867,
  1933341,
  946209,
  1587234,
  292899,
  100388,
  2486310,
  2375719,
  2117672,
  2170920,
  1728552,
  1118246,
  2375724,
  1138736,
  2244658,
  661559,
  2486328,
  2457658,
  2887739,
  2439228,
  557114,
  2457659,
  2457662,
  2117694,
  1972288,
  1632324,
  1951822,
  1728590,
  1642580,
  1435733,
  2469974,
  2138199,
  1124440,
  1368154,
  1642587,
  2928732,
  751709,
  925788,
  491610,
  2408551,
  1622121,
  1499242,
  1319025,
  1175666,
  2431093,
  1124470,
  2431097,
  2762873,
  2693250,
  1435778,
  2105486,
  1886353,
  1886354,
  77971,
  2576531,
  77970,
  1886359,
  1343641,
  2470041,
  333979,
  1886364,
  1124509,
  1319068,
  333980,
  333984,
  2689185,
  440481,
  2570395,
  520359,
  520363,
  1812654,
  2470063,
  1886383,
  333999,
  252088,
  1142969,
  610493,
  2470077,
  776388,
  2650308,
  103444

In [39]:
np.asarray(feat_ext(dict_aid_post[292868])).shape

(1, 373, 768)

In [41]:
len(feat_ext(dict_aid_post[292868])[0][0])

768